TSG022 - Get external IP address for kubeadm host
=================================================

Description
-----------

Get the external IP address of the host of kubeadm

Steps
-----

### Common functions

Define helper functions used in this notebook.

In [ ]:
# Define `run` function for transient fault handling, suggestions on error, and scrolling updates on Windows
import sys
import os
import re
import platform
import shlex
import shutil
import datetime

from subprocess import Popen, PIPE
from IPython.display import Markdown

def run(cmd, return_output=False, no_output=False, error_hints=[], retry_hints=[], retry_count=0):
    """
    Run shell command, stream stdout, print stderr and optionally return output
    """
    max_retries = 5
    install_hint = None
    output = ""
    retry = False

    # shlex.split is required on bash and for Windows paths with spaces
    #
    cmd_actual = shlex.split(cmd)

    # When running python, use the python in the ADS sandbox ({sys.executable})
    #
    if cmd.startswith("python "):
        cmd_actual[0] = cmd_actual[0].replace("python", sys.executable)

        # On Mac, when ADS is not launched from terminal, LC_ALL may not be set, which causes pip installs to fail
        # with:
        #
        #       UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 4969: ordinal not in range(128)
        #
        # Setting it to a default value of "en_US.UTF-8" enables pip install to complete
        #
        if platform.system() == "Darwin" and "LC_ALL" not in os.environ:
            os.environ["LC_ALL"] = "en_US.UTF-8"

        python_retry_hints, python_error_hints, install_hint = python_hints()
        retry_hints += python_retry_hints
        error_hints += python_error_hints

    if (cmd.startswith("kubectl ")):
        kubectl_retry_hints, kubectl_error_hints, install_hint = kubectl_hints()
        retry_hints += kubectl_retry_hints
        error_hints += kubectl_error_hints

    if (cmd.startswith("azdata ")):
        azdata_retry_hints, azdata_error_hints, install_hint = azdata_hints()
        retry_hints += azdata_retry_hints
        error_hints += azdata_error_hints

    # Find the path based location (shutil.which) of the executable that will be run (and display it to aid supportability), this
    # seems to be required for .msi installs of azdata.cmd/az.cmd.  (otherwise Popen returns FileNotFound) 
    #
    # NOTE: Bash needs cmd to be the list of the space separated values hence shlex.split.
    #
    which_binary = shutil.which(cmd_actual[0])

    if which_binary == None:
        if install_hint is not None:
            display(Markdown(f'SUGGEST: Use {install_hint} to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)")
    else:   
        cmd_actual[0] = which_binary

    start_time = datetime.datetime.now().replace(microsecond=0)

    print(f"START: {cmd} @ {start_time} ({datetime.datetime.utcnow().replace(microsecond=0)} UTC)")
    print(f"       using: {which_binary} ({platform.system()} {platform.release()} on {platform.machine()})")
    print(f"       cwd: {os.getcwd()}")

    # Command-line tools such as CURL and AZDATA HDFS commands output
    # scrolling progress bars, which causes Jupyter to hang forever, to
    # workaround this, use no_output=True
    #
    try:
        if no_output:
            p = Popen(cmd_actual)
        else:
            p = Popen(cmd_actual, stdout=PIPE, stderr=PIPE, bufsize=1)
            with p.stdout:
                for line in iter(p.stdout.readline, b''):
                    line = line.decode()
                    if return_output:
                        output = output + line
                    else:
                        if cmd.startswith("azdata notebook run"): # Hyperlink the .ipynb file
                            regex = re.compile('  "(.*)"\: "(.*)"') 
                            match = regex.match(line)
                            if match:
                                if match.group(1).find("HTML") != -1:
                                    display(Markdown(f' - "{match.group(1)}": "{match.group(2)}"'))
                                else:
                                    display(Markdown(f' - "{match.group(1)}": "[{match.group(2)}]({match.group(2)})"'))
                        else:
                            print(line, end='')
        p.wait()
    except FileNotFoundError as e:
        if install_hint is not None:
            display(Markdown(f'SUGGEST: Use {install_hint} to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)") from e

    if not no_output:
        for line in iter(p.stderr.readline, b''):
            line_decoded = line.decode()

            # azdata emits a single empty line to stderr when doing an hdfs cp, don't
            # print this empty "ERR:" as it confuses.
            #
            if line_decoded == "":
                continue
            
            print(f"ERR: {line_decoded}", end='')

            for error_hint in error_hints:
                if line_decoded.find(error_hint[0]) != -1:
                    display(Markdown(f'SUGGEST: Use [{error_hint[2]}]({error_hint[1]}) to resolve this issue.'))

            for retry_hint in retry_hints:
                if line_decoded.find(retry_hint) != -1:
                    if retry_count < max_retries:
                        print(f"RETRY: {retry_count} (due to: {retry_hint})")
                        retry_count = retry_count + 1
                        output = run(cmd, return_output=return_output, error_hints=error_hints, retry_hints=retry_hints, retry_count=retry_count)

                        if return_output:
                            return output
                        else:
                            return

    elapsed = datetime.datetime.now().replace(microsecond=0) - start_time

    if p.returncode != 0:
        raise SystemExit(f'Shell command:\n\n\t{cmd} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(p.returncode)}.\n')

    print(f'\nSUCCESS: {elapsed}s elapsed\n')

    if return_output:
        return output

def kubectl_hints():

    retry_hints = [
        "A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond"
    ]

    error_hints = [
    ["""no such host""", """../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb""", """TSG010 - Get configuration contexts"""],
    ["""no such host""", """../repair/tsg011-restart-sparkhistory-server.ipynb""", """TSG011 - Restart sparkhistory server"""],
    ["""No connection could be made because the target machine actively refused it""", """../repair/tsg056-kubectl-no-connection-could-be-made.ipynb""", """TSG056 - Kubectl fails with No connection could be made because the target machine actively refused it"""]
]

    install_hint = "[SOP036 - Install kubectl command line interface](../install/sop036-install-kubectl.ipynb)'"

    return retry_hints, error_hints, install_hint


print('Common functions defined successfully.')

### Get IP address

In [ ]:
run('kubectl get node --selector="node-role.kubernetes.io/master" -o=jsonpath="{.items[0].status.addresses[0].address}"')

In [ ]:
print('Notebook execution complete.')